## Estrutura para coletar os dados

```
    semester_id
             |--- class_id
             |        |--- user_id
             |        |      |--- {feature1 : feature_value}
             |        |      |--- {feature2 : feature_value}
             |        |      |--- .......
             |        |      |--- {featureN : feature_value}
```

## Criando a estrutura para armazenar os dados dos alunos

### Imports

In [ ]:
import os
import re
import pymysql

from pprint import pprint

### Extraindo os semestres

In [ ]:
ROOT = '../dataset/workbench/'
database = {}
for semester_id in os.listdir(ROOT):
    database[semester_id] = {}

pprint(database)

### Extraindo as turmas

In [ ]:
ROOT = '../dataset/workbench/{semester}'
for semester_id in database.keys():
    path = ROOT.replace('{semester}', semester_id)
    classes = os.listdir(path)
    for class_id in classes:
        database[semester_id][class_id] = {}


### Extraindo os usuarios

In [ ]:
ROOT = '../dataset/workbench/{semester}/{class}/users/'
for semester_id in database.keys():
    for class_id in database[semester_id].keys():
        path = ROOT.replace('{semester}', semester_id)
        path = path.replace('{class}', class_id)
        for user_id in os.listdir(path):
            database[semester_id][class_id][user_id] = {}

### Acrescentando na estrutura as caracteristicas dos alunos

In [ ]:
features_aluno = [
  'user_id',
  'semester_id',
  'class_number',
  'course_id',
  'course_name',
  'inst_id_curr',
  'inst_name_curr',
  'high_school_name',
  'school_type',
  'shift',
  'graduation_year',
  'has_pc',
  'share_pc',
  'has_internet',
  'speed_internet',
  'prev_exp_lang',
  'language',
  'worked',
  'company_name',
  'year_started_work',
  'year_stoped_work',
  'other_degree',
  'inst_name_prev',
  'name_other_degree',
  'year_started_prev_degree',
  'year_stoped_prev_degree',
  'sex',
  'year_birth',
  'civil_status',
  'have_kids',
]

for semester_id in database.keys():
    for class_id in database[semester_id].keys():
        for user_id in database[semester_id][class_id].keys():
            for feature in features_aluno:
                database[semester_id][class_id][user_id][feature] = 'NULL'
                database[semester_id][class_id][user_id]['semester_id'] = semester_id
                database[semester_id][class_id][user_id]['class_number'] = class_id

### Preenchendo a estrutura com os dados dos alunos

In [ ]:

ROOT = '../dataset/workbench/{semester_id}/{class_id}/users/{user_id}'

'''
features_text = [
    'course id',#0
    'course name',#1
    'institution id',#2
    'institution name',#3
    'high school name',#4
    'school type',#5
    'shift',#6
    'graduation year',#7
    'has a PC',#8
    'share this PC', #9
    'this PC has access to',#10
    'previous experience',#11
    'worked or interned',#12
    'company name',#13
    'year started working',#14
    'year stopped working',#15
    'started other degree',#16
    'degree course',#17
    'year started this',#18
    'year stopped this',#19
    'sex',#20
    'yaer of birth',#21
    'civil status',#22
    'have kids'#23
]
'''

for semester_id in database.keys():
    for class_id in database[semester_id].keys():
        for user_id in database[semester_id][class_id].keys():
            path = ROOT.replace('{semester_id}', semester_id)
            path = path.replace('{class_id}', class_id)
            path = path.replace('{user_id}', user_id)
         
            path_file = os.path.join(path, 'user.data')
            is_curr_inst = False
            
            with open(path_file, 'r') as fl:
                for linha in fl:
                    if 'course id' in linha:
                        linha = linha.split(':')[1]
                        course_id = linha.split('\n')[0][1:]
                        if len(course_id) > 0 and 'n/a' not in course_id:
                            database[semester_id][class_id][user_id]['course_id'] = course_id
                        
                    elif 'course name' in linha:
                        linha = linha.split(':')[1]
                        course_name = linha.split('\n')[0][1:]
                        if len(course_name) > 0 and 'n/a' not in course_name:
                            database[semester_id][class_id][user_id]['course_name'] = course_name
                              
                    elif 'institution id' in linha:
                        linha = linha.split(':')[1]
                        inst_id_curr = linha.split('\n')[0][1:]
                        if len(inst_id_curr) > 0 and 'n/a' not in inst_id_curr:
                            database[semester_id][class_id][user_id]['inst_id_curr'] = inst_id_curr
                    
                    elif 'institution name' in linha:
                        linha = linha.split(':')[1]
                        inst_name_curr = linha.split('\n')[0][1:]
                        if is_curr_inst == False:
                            is_curr_inst = True
                            if len(inst_name_curr) > 0 and 'n/a' not in inst_name_curr:
                                database[semester_id][class_id][user_id]['inst_name_curr'] = inst_name_curr
                        else:
                            if len(inst_name_curr) > 0 and 'n/a' not in inst_name_curr:
                                database[semester_id][class_id][user_id]['inst_name_prev'] = inst_name_curr                            
                    
                    elif 'high school name' in linha:
                        linha = linha.split(':')[1]
                        high_school_name = linha.split('\n')[0][1:]
                        if len(high_school_name) > 0 and 'n/a' not in high_school_name:
                            find_p = re.search(r"\w'\w", high_school_name)
                            if find_p:
                                # alterando o nome sant'ana para santa Ana
                                high_school_name = high_school_name.replace(find_p.group(0), 'ta A')
                            f = re.search(r"\w'", high_school_name)
                            if f:
                                # alterando o nome lalaland' para lalaland
                                high_school_name = high_school_name.replace(f.group(0), 'd')
                            database[semester_id][class_id][user_id]['high_school_name'] = high_school_name
                            
                    elif 'school type' in linha:
                        linha = linha.split(':')[1]
                        school_type = linha.split('\n')[0][1:]
                        if len(school_type) > 0 and 'n/a' not in school_type:
                            database[semester_id][class_id][user_id]['school_type'] = school_type
                        
                    elif 'shift' in linha:
                        linha = linha.split(':')[1]
                        shift = linha.split('\n')[0][1:]
                        if len(shift) > 0 and 'n/a' not in shift:
                            database[semester_id][class_id][user_id]['shift'] = shift
                        
                    elif 'graduation year' in linha:
                        linha = linha.split(':')[1]
                        graduation_year = linha.split('\n')[0][1:]
                        if len(graduation_year) > 0 and 'n/a' not in graduation_year:
                            database[semester_id][class_id][user_id]['graduation_year'] = graduation_year
                        
                    elif 'has a PC' in linha:
                        linha = linha.split(':')[1]
                        has_pc = linha.split('\n')[0][1:]
                        if len(has_pc) > 0 and 'n/a' not in has_pc:
                            database[semester_id][class_id][user_id]['has_pc'] = has_pc
                    
                    elif 'share this PC' in linha:
                        linha = linha.split(':')[1]
                        share_pc = linha.split('\n')[0][1:]
                        if len(share_pc) > 0 and 'n/a' not in share_pc:
                            database[semester_id][class_id][user_id]['share_pc'] = share_pc
                    
                    elif 'this PC has access to' in linha:
                        linha = linha.split(':')[1]
                        has_internet = linha.split('\n')[0][1:]
                        if len(has_internet) > 0 and 'n/a' not in has_internet:
                            if 'no' in has_internet:
                                database[semester_id][class_id][user_id]['has_internet'] = has_internet
                            else:
                                has = has_internet.split(' ')[0]
                                database[semester_id][class_id][user_id]['has_internet'] = has
                                speed = has_internet.split(' ')[1][1:]
                                database[semester_id][class_id][user_id]['speed_internet'] = speed
                    
                    elif 'previous experience' in linha:
                        linha = linha.split(':')[1]
                        prev_exp_lang = linha.split('\n')[0][1:]
                        if len(prev_exp_lang) > 0 and 'n/a' not in prev_exp_lang:
                            if 'no' in prev_exp_lang:
                                database[semester_id][class_id][user_id]['prev_exp_lang'] = prev_exp_lang
                            else:
                                prev = prev_exp_lang.split('(')[0][:-1]
                                database[semester_id][class_id][user_id]['prev_exp_lang'] = prev
                                lang = prev_exp_lang.split('(')[1][:-1]
                                database[semester_id][class_id][user_id]['language'] = lang
                    
                    elif 'worked or interned' in linha:
                        linha = linha.split(':')[1]
                        worked = internet = linha.split('\n')[0][1:]
                        if len(worked) > 0 and 'n/a' not in worked:
                            database[semester_id][class_id][user_id]['worked'] = worked
                    
                    elif 'company name' in linha:
                        linha = linha.split(':')[1]
                        company_name = linha.split('\n')[0][1:]
                        if len(company_name) > 0 and 'n/a' not in company_name:
                            database[semester_id][class_id][user_id]['company_name'] = company_name
                            
                    elif 'year started working' in linha:
                        linha = linha.split(':')[1]
                        year_started_work = linha.split('\n')[0][1:]
                        if len(year_started_work) > 0 and 'n/a' not in year_started_work:
                            database[semester_id][class_id][user_id]['year_started_work'] = year_started_work
                    
                    elif 'year stopped working' in linha:
                        linha = linha.split(':')[1]
                        year_stoped_work = linha.split('\n')[0][1:]
                        if len(year_stoped_work) > 0 and 'n/a' not in year_stoped_work:
                            database[semester_id][class_id][user_id]['year_stoped_work'] = year_stoped_work
                    
                    elif 'started other degree' in linha:
                        linha = linha.split(':')[1]
                        other_degree = linha.split('\n')[0][1:]
                        if len(other_degree) > 0 and 'n/a' not in other_degree:
                            database[semester_id][class_id][user_id]['other_degree'] = other_degree
                    
                    elif 'degree course' in linha:
                        linha = linha.split(':')[1]
                        name_other_degree = linha.split('\n')[0][1:]
                        if len(name_other_degree) > 0 and 'n/a' not in name_other_degree:
                            database[semester_id][class_id][user_id]['name_other_degree'] = name_other_degree
                    
                    elif 'year started this' in linha:
                        linha = linha.split(':')[1]
                        year_started_prev_degree = linha.split('\n')[0][1:]
                        if len(year_started_prev_degree) > 0 and 'n/a' not in year_started_prev_degree:
                            database[semester_id][class_id][user_id]['year_started_prev_degree'] = year_started_prev_degree
                    
                    elif 'year stopped this' in linha:
                        linha = linha.split(':')[1]
                        year_stoped_prev_degree = linha.split('\n')[0][1:]
                        if len(year_stoped_prev_degree) > 0 and 'n/a' not in year_stoped_prev_degree:
                            database[semester_id][class_id][user_id]['year_stoped_prev_degree'] = year_stoped_prev_degree
                    
                    elif 'sex' in linha:
                        linha = linha.split(':')[1]
                        sex = linha.split('\n')[0][1:]
                        if len(sex) > 0 and 'n/a' not in sex:
                            database[semester_id][class_id][user_id]['sex'] = sex
                            
                    elif 'year of birth' in linha:
                        linha = linha.split(':')[1]
                        year_birth = linha.split('\n')[0][1:]
                        if len(year_birth) > 0 and 'n/a' not in year_birth:
                            database[semester_id][class_id][user_id]['year_birth'] = year_birth
                    
                    elif 'civil status' in linha:
                        linha = linha.split(':')[1]
                        civil_status = linha.split('\n')[0][1:]
                        if len(civil_status) > 0 and 'n/a' not in civil_status:
                            database[semester_id][class_id][user_id]['civil_status'] = civil_status
                    
                    elif 'have kids' in linha:
                        linha = linha.split(':')[1]
                        have_kids = linha.split('\n')[0][1:]
                        if len(have_kids) > 0 and'n/a' not in have_kids:
                            database[semester_id][class_id][user_id]['have_kids'] = have_kids


## Inserindo os dados no banco

### Estabelecendo a conexao

In [ ]:
def conn():
    connection = pymysql.connect(host='localhost',
                             user='root',
                             password='open',
                             db='dataset-workbench',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    return connection

### Inserindo dados no banco

In [ ]:
#connetion.close()
connection = conn()

cursor = connection.cursor()

varchar_features = [
  'user_id',
  'semester_id',
  'course_name',
  'inst_name_curr',
  'high_school_name',
  'school_type',
  'shift',
  'has_pc',
  'share_pc',
  'has_internet',
  'speed_internet',
  'prev_exp_lang',
  'language',
  'worked',
  'company_name',
  'year_started_work',
  'year_stoped_work',
  'other_degree',
  'inst_name_prev',
  'name_other_degree',
  'sex',
  'civil_status',
  'have_kids',
]

def is_varchar(feature):
    if feature in varchar_features and value != 'NULL':
        return True
    else: return False

for semester_id in database.keys():
    for class_id in database[semester_id].keys():
        for user_id in database[semester_id][class_id].keys():
            
            values = ''
            features = ''
            insert_user_sql = ''
            
            for feature in database[semester_id][class_id][user_id].keys():
                value = database[semester_id][class_id][user_id][feature]
                features += f'{feature}, '
                if 'user_id' in feature:
                    value = f'{semester_id}#{class_id}#{user_id}'
                if is_varchar(feature):
                    values += f"'{value}', "
                else:
                    values += f"{value}, "

            insert_user_sql = f'INSERT INTO user({features[:-2]}) VALUES ({values[:-2]});'
            print(insert_user_sql)
            cursor.execute(insert_user_sql)
connection.commit()                
connection.close()       